In [120]:
import pandas as pd
import xgboost as xgb #Modlo xgboost
from sklearn.model_selection import train_test_split #Faz split de treino e teste
from sklearn.metrics import accuracy_score #Mostra acurracy
from sklearn.preprocessing import LabelEncoder #Transforma dados não numericos em numericos e vice-versa
from ydata_profiling import ProfileReport as PR

In [121]:
'''
    1. top-left-square: {x,o,b}
    2. top-middle-square: {x,o,b}
    3. top-right-square: {x,o,b}
    4. middle-left-square: {x,o,b}
    5. middle-middle-square: {x,o,b}
    6. middle-right-square: {x,o,b}
    7. bottom-left-square: {x,o,b}
    8. bottom-middle-square: {x,o,b}
    9. bottom-right-square: {x,o,b}
   10. Class: {positive,negative}
'''

'\n    1. top-left-square: {x,o,b}\n    2. top-middle-square: {x,o,b}\n    3. top-right-square: {x,o,b}\n    4. middle-left-square: {x,o,b}\n    5. middle-middle-square: {x,o,b}\n    6. middle-right-square: {x,o,b}\n    7. bottom-left-square: {x,o,b}\n    8. bottom-middle-square: {x,o,b}\n    9. bottom-right-square: {x,o,b}\n   10. Class: {positive,negative}\n'

In [122]:
# Tratamento: Adicionei nomes às colunas no csv(Primeira linha)
# Troquei positive/negative por X Ganhou/Tem jogo
# Selecionei somente os valores X positive e inverti x por o para criar o "O Ganhou"
# Tirei todos os valores iguais entre "O ganhou" e "Tem jogo"
# Separei todos os valores sem nenhum espaço "b" do "Tem jogo" e classifiquei como empate
# O df result é o resultado de todas as operações de limpeza, ele aumenta o tamanho do conjunto de dados
# pois ignora que o lado 'O' é o segundo jogador

In [123]:
df = pd.read_csv('tic+tac+toe+endgame/tic-tac-toe.data')

In [124]:
# Tornando positivo e negativo dados númericos
def rename(x):
    if x == 'positive':
        return 'X ganhou'
    else:
        return 'Tem Jogo'
    
df['Xwin'] = df['Xwin'].map(rename)

In [125]:
label = LabelEncoder()

In [126]:
onlyX = df[df['Xwin'] == 'X ganhou']

In [127]:
temp = onlyX.replace('x','temp')
temp = temp.replace('o','x')
onlyO = temp.replace('temp','o')
onlyO = onlyO.replace('X ganhou','O ganhou')
display(onlyO)

,1,2,3,4,5,6,7,8,9,Xwin
0,o,o,o,o,x,x,o,x,x,O ganhou
1,o,o,o,o,x,x,x,o,x,O ganhou
2,o,o,o,o,x,x,x,x,o,O ganhou
3,o,o,o,o,x,x,x,b,b,O ganhou
4,o,o,o,o,x,x,b,x,b,O ganhou
...,...,...,...,...,...,...,...,...,...,...
621,b,b,b,o,o,o,x,b,x,O ganhou
622,b,b,b,o,o,o,b,x,x,O ganhou
623,b,b,b,x,x,b,o,o,o,O ganhou
624,b,b,b,x,b,x,o,o,o,O ganhou


In [128]:
df_all = df[df['Xwin']=='Tem Jogo'].merge(onlyO.drop_duplicates(), on = ['1','2','3','4','5','6','7','8','9'], 
                   how='left', indicator=True)
df_all

,1,2,3,4,5,6,7,8,9,Xwin_x,Xwin_y,_merge
0,x,x,o,x,x,o,o,b,o,Tem Jogo,NaN,left_only
1,x,x,o,x,x,o,b,o,o,Tem Jogo,NaN,left_only
2,x,x,o,x,x,b,o,o,o,Tem Jogo,NaN,left_only
3,x,x,o,x,o,x,o,o,b,Tem Jogo,NaN,left_only
4,x,x,o,x,o,x,o,b,o,Tem Jogo,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...
327,o,x,x,x,o,o,o,x,x,Tem Jogo,NaN,left_only
328,o,x,o,x,x,o,x,o,x,Tem Jogo,NaN,left_only
329,o,x,o,x,o,x,x,o,x,Tem Jogo,NaN,left_only
330,o,x,o,o,x,x,x,o,x,Tem Jogo,NaN,left_only


In [129]:
noWin = df_all[df_all['_merge'] == 'left_only']

In [130]:
noWin.drop(['Xwin_y','_merge'],axis=1,inplace=True)
noWin

,1,2,3,4,5,6,7,8,9,Xwin_x
0,x,x,o,x,x,o,o,b,o,Tem Jogo
1,x,x,o,x,x,o,b,o,o,Tem Jogo
2,x,x,o,x,x,b,o,o,o,Tem Jogo
3,x,x,o,x,o,x,o,o,b,Tem Jogo
4,x,x,o,x,o,x,o,b,o,Tem Jogo
...,...,...,...,...,...,...,...,...,...,...
327,o,x,x,x,o,o,o,x,x,Tem Jogo
328,o,x,o,x,x,o,x,o,x,Tem Jogo
329,o,x,o,x,o,x,x,o,x,Tem Jogo
330,o,x,o,o,x,x,x,o,x,Tem Jogo


In [132]:
#df = df.append(onlyO, ignore_index=True)
#display(df)
#df.loc[(df['column_name'] >= A) & (df['column_name'] <= B)]

empate = noWin[(noWin['1'] != 'b') &
          (noWin['2'] != 'b') &
          (noWin['3'] != 'b') &
          (noWin['4'] != 'b') &
          (noWin['5'] != 'b') &
          (noWin['6'] != 'b') &
          (noWin['7'] != 'b') &
          (noWin['8'] != 'b') &
          (noWin['9'] != 'b')]
empate['Xwin_x'].replace('Tem Jogo','Empate',inplace = True)
empate.columns = ['1','2','3','4','5','6','7','8','9','Xwin']
empate

/tmp/ipykernel_91475/3666853286.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empate['Xwin_x'].replace('Tem Jogo','Empate',inplace = True)


,1,2,3,4,5,6,7,8,9,Xwin
316,x,x,o,o,x,x,x,o,o,Empate
317,x,x,o,o,o,x,x,x,o,Empate
318,x,x,o,o,o,x,x,o,x,Empate
319,x,o,x,x,x,o,o,x,o,Empate
320,x,o,x,x,o,x,o,x,o,Empate
321,x,o,x,x,o,o,o,x,x,Empate
322,x,o,x,o,x,x,o,x,o,Empate
323,x,o,x,o,o,x,x,x,o,Empate
324,x,o,o,o,x,x,x,x,o,Empate
325,o,x,x,x,x,o,o,o,x,Empate


In [142]:
noWin.drop(list(empate.index),inplace=True)
noWin

,1,2,3,4,5,6,7,8,9,Xwin_x
0,x,x,o,x,x,o,o,b,o,Tem Jogo
1,x,x,o,x,x,o,b,o,o,Tem Jogo
2,x,x,o,x,x,b,o,o,o,Tem Jogo
3,x,x,o,x,o,x,o,o,b,Tem Jogo
4,x,x,o,x,o,x,o,b,o,Tem Jogo
...,...,...,...,...,...,...,...,...,...,...
311,b,b,o,x,o,x,o,b,x,Tem Jogo
312,b,b,o,x,o,b,o,x,x,Tem Jogo
313,b,b,o,x,b,o,x,x,o,Tem Jogo
314,b,b,o,b,x,o,x,x,o,Tem Jogo


In [144]:
onlyX

,1,2,3,4,5,6,7,8,9,Xwin
0,x,x,x,x,o,o,x,o,o,X ganhou
1,x,x,x,x,o,o,o,x,o,X ganhou
2,x,x,x,x,o,o,o,o,x,X ganhou
3,x,x,x,x,o,o,o,b,b,X ganhou
4,x,x,x,x,o,o,b,o,b,X ganhou
...,...,...,...,...,...,...,...,...,...,...
621,b,b,b,x,x,x,o,b,o,X ganhou
622,b,b,b,x,x,x,b,o,o,X ganhou
623,b,b,b,o,o,b,x,x,x,X ganhou
624,b,b,b,o,b,o,x,x,x,X ganhou


In [145]:
onlyO

,1,2,3,4,5,6,7,8,9,Xwin
0,o,o,o,o,x,x,o,x,x,O ganhou
1,o,o,o,o,x,x,x,o,x,O ganhou
2,o,o,o,o,x,x,x,x,o,O ganhou
3,o,o,o,o,x,x,x,b,b,O ganhou
4,o,o,o,o,x,x,b,x,b,O ganhou
...,...,...,...,...,...,...,...,...,...,...
621,b,b,b,o,o,o,x,b,x,O ganhou
622,b,b,b,o,o,o,b,x,x,O ganhou
623,b,b,b,x,x,b,o,o,o,O ganhou
624,b,b,b,x,b,x,o,o,o,O ganhou


In [148]:
noWin.columns = ['1','2','3','4','5','6','7','8','9','Xwin']
noWin

,1,2,3,4,5,6,7,8,9,Xwin
0,x,x,o,x,x,o,o,b,o,Tem Jogo
1,x,x,o,x,x,o,b,o,o,Tem Jogo
2,x,x,o,x,x,b,o,o,o,Tem Jogo
3,x,x,o,x,o,x,o,o,b,Tem Jogo
4,x,x,o,x,o,x,o,b,o,Tem Jogo
...,...,...,...,...,...,...,...,...,...,...
311,b,b,o,x,o,x,o,b,x,Tem Jogo
312,b,b,o,x,o,b,o,x,x,Tem Jogo
313,b,b,o,x,b,o,x,x,o,Tem Jogo
314,b,b,o,b,x,o,x,x,o,Tem Jogo


In [147]:
empate

,1,2,3,4,5,6,7,8,9,Xwin
316,x,x,o,o,x,x,x,o,o,Empate
317,x,x,o,o,o,x,x,x,o,Empate
318,x,x,o,o,o,x,x,o,x,Empate
319,x,o,x,x,x,o,o,x,o,Empate
320,x,o,x,x,o,x,o,x,o,Empate
321,x,o,x,x,o,o,o,x,x,Empate
322,x,o,x,o,x,x,o,x,o,Empate
323,x,o,x,o,o,x,x,x,o,Empate
324,x,o,o,o,x,x,x,x,o,Empate
325,o,x,x,x,x,o,o,o,x,Empate


In [149]:
result = onlyX.append(onlyO, ignore_index=True)
result = result.append(noWin,ignore_index=True)
result = result.append(empate,ignore_index=True)
result

/tmp/ipykernel_91475/4091617420.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = onlyX.append(onlyO, ignore_index=True)
/tmp/ipykernel_91475/4091617420.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(noWin,ignore_index=True)
/tmp/ipykernel_91475/4091617420.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(empate,ignore_index=True)


,1,2,3,4,5,6,7,8,9,Xwin
0,x,x,x,x,o,o,x,o,o,X ganhou
1,x,x,x,x,o,o,o,x,o,X ganhou
2,x,x,x,x,o,o,o,o,x,X ganhou
3,x,x,x,x,o,o,o,b,b,X ganhou
4,x,x,x,x,o,o,b,o,b,X ganhou
...,...,...,...,...,...,...,...,...,...,...
1579,o,x,x,x,o,o,o,x,x,Empate
1580,o,x,o,x,x,o,x,o,x,Empate
1581,o,x,o,x,o,x,x,o,x,Empate
1582,o,x,o,o,x,x,x,o,x,Empate


In [150]:
#result.to_csv('TicTacLimpo.csv')